# 1. 라이브러리 import

## 1) 모델 관련

In [1]:
import os

# 음성 데이터 처리
import librosa
import numpy as np
import pandas as pd

# 데이터 시각화
from matplotlib import pyplot as plt

# 모델 관련
import sklearn
from sklearn import preprocessing # AttributeError: module 'sklearn' has no attribute 'preprocessing'
import tensorflow as tf

2024-10-11 07:37:49.061689: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-11 07:37:49.081028: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-11 07:37:49.087786: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-11 07:37:49.101918: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## 2) 모델 사용을 위한 라이브러리

In [2]:
import import_ipynb
import voice_strength

importing Jupyter notebook from voice_strength.ipynb


I0000 00:00:1728599871.105599 1628581 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728599871.174488 1628581 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728599871.175159 1628581 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1728599871.327202 1628581 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Num GPUs Available:  1


## 3) 플라스크 서버 구축을 위한 라이브러리

In [3]:
import tempfile
import logging
from flask import Flask, request, jsonify
import torch
import threading
import queue
import requests
from flask_restx import Api, Resource, fields, marshal
import signal
from datetime import datetime

# 2. GPU 사용 확인

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

Num GPUs Available:  1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# 3. 이것 저것 설정

In [5]:
torch.cuda.empty_cache()

# 4. 서버 구축

## 1) 로그 설정

In [6]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## 2) 라우터 및 swagger 설정

In [7]:
app = Flask(__name__)

# swagger (http://127.0.0.1:5000/api-docs)
api = Api(
    app,
    version='1.0',
    title='발화 강세 분석 AI를 위한 API 문서',
    description='Swagger 문서',
    doc="/api-docs"
)

voice_api = api.namespace(name='voice', description='발화 분석 관련 API')

## 3) dto 정의

In [8]:
analysis_request_model = api.model('request', {
    'noteId': fields.Integer(readOnly=True, required=True, description='노트 id'), 
    'objectUrl': fields.String(required=True, description='녹음본이 저장된 위치'),
    'processId': fields.String(required=True, description='각 요청에 대한 UUID'),
})

analysis_response_model = api.model('response', {
    'noteId': fields.String(readOnly=True, required=True, description='노트 id'),
    'processId': fields.String(required=True, description='각 요청에 대한 UUID'),
    'status': fields.String(required=True, description='처리 상태'),
})

analysis_response_data = {
    'noteId': 0,
    'processId': '',
    'status': '',
}

base_response = {
    'code': 200,
    'message': '',
    'data': None
}

In [9]:
process_request_queue = []

## 4) 컨트롤러

In [10]:
@voice_api.route('/')
class Index(Resource):
    def get(self):
        base_response['code'] = 200
        base_response['message'] = 'hello'
        base_response['data'] = '안녕하'

        return jsonify(base_response)

In [11]:
@voice_api.route('/analysis')
class Analysis(Resource):
    @voice_api.expect(analysis_request_model)  # 요청 모델을 Swagger에 추가
    # @api.marshal_with(analysis_response_model)  # 응답 모델을 Swagger에 추가

    def post(self):
        try:
            req = marshal(request.get_json(force=True), analysis_request_model)
            print(f"{datetime.now().time()} {req}")
            
            # 유효 범위 처리
            if req['noteId'] < 0:
                return


            audio_url = req['objectUrl']
            
            if audio_url.strip() != '':
                try:
                    print(f"{datetime.now().time()} Downloading audio file from URI: {audio_url}")
                    logger.info(f"{datetime.now().time()} Downloading audio file from URI: {audio_url}")
                    
                    response = requests.get(audio_url) # .wav 파일 다운로드
                    response.raise_for_status()  # 오류 발생 시 예외 처리
                    
                    # 오디오 데이터를 저장할 임시 파일 생성
                    with tempfile.NamedTemporaryFile(delete=False) as temp_audio:
                        temp_audio.write(response.content)  # 다운로드한 파일 내용을 임시 파일에 저장
                        temp_audio_path = temp_audio.name  # 임시 파일 경로 저장

                    print(f"{datetime.now().time()} Audio file downloaded and saved to {temp_audio_path}")
                    logger.info(f"{datetime.now().time()}  Audio file downloaded and saved to {temp_audio_path}")



                    # time_list = send_analysis_result(req['noteId'], req['processId'], temp_audio_path)
                    # analysis_response_data['noteId'] = req['noteId']
                    # analysis_response_data['processId'] = req['processId']
                    # analysis_response_data['status'] = time_list
                    
                    # 비동기로 음성 분석 실행
                    task_thread = threading.Thread(target=send_analysis_result, args=(req['noteId'], req['processId'], temp_audio_path,))
                    task_thread.start()
                    
                    # 작업 ID와 상태 반환
                    analysis_response_data['noteId'] = req['noteId']
                    analysis_response_data['processId'] = req['processId']
                    analysis_response_data['status'] = 'processing'
                    

                    return analysis_response_data, 202
                    
                except requests.exceptions.RequestException as e:
                    logger.error(f"Failed to download audio file: {str(e)}")
                    return jsonify({'error': 'Failed to download audio file'}), 400

        except ValueError as e:
            base_response['code'] = 400
            base_response['message'] = 'error: ' + str(e)
            return jsonify(base_response)
        
        except Exception as e:
            base_response['code'] = 500
            base_response['message'] = 'error: ' + str(e)
            return jsonify(base_response)

## 5) 기타 메서드

In [12]:
# 비동기 작업
def send_analysis_result(noteId, processId, audio_path):
    second_list = voice_strength.get_abnormal_interval('', [audio_path])

    data = {
        'id': int(noteId),
        'process_id': str(processId),
        'anomaly_time': second_list
    }
    
    # 스프링의 /voice/analysisResult api 호출
    try:
        response = requests.post("http://REMOVED:8080/voice/analysisResult", json=data)
        print(f"{datetime.now().time()} request: {data}")
        print(f"{datetime.now().time()} responst: {response}")
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"{datetime.now().time()} 결과 전송 오류: {e}")

In [13]:
def denoise(audio_data):
    return enhance(dfn_model, df_state, audio_data)

## 6) 에러 핸들링

In [14]:
@app.errorhandler(405)
def internal_error(error):
    logger.error(f"Method Not Allowed: {str(error)}")

    base_response['code'] = 405
    base_response['message'] = 'Method Not Allowed'

    return jsonify(base_response)

In [15]:
@app.errorhandler(500)
def internal_error(error):
    logger.error(f"Internal Server Error: {str(error)}")
    return {"error": "Internal Server Error"}, 500

## 7) main

In [ ]:
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=4998)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:4998
 * Running on http://70.12.130.111:4998
INFO:werkzeug:Press CTRL+C to quit
INFO:__main__:08:03:25.031426 Downloading audio file from URI: https://timeisnullnull.s3.ap-northeast-2.amazonaws.com/wav/fa8d5ab6-5e49-43dd-911c-742f1980f6fe.wav
INFO:__main__:08:03:25.173789  Audio file downloaded and saved to /tmp/tmpqdsytb_b
INFO:werkzeug:127.0.0.1 - - [11/Oct/2024 08:03:25] "POST /voice/analysis HTTP/1.1" 202 -


08:03:25.030837 {'noteId': 47, 'objectUrl': 'https://timeisnullnull.s3.ap-northeast-2.amazonaws.com/wav/fa8d5ab6-5e49-43dd-911c-742f1980f6fe.wav', 'processId': '833e0966-da63-4829-a69e-517c6db6a469'}
08:03:25.031405 Downloading audio file from URI: https://timeisnullnull.s3.ap-northeast-2.amazonaws.com/wav/fa8d5ab6-5e49-43dd-911c-742f1980f6fe.wav
08:03:25.173748 Audio file downloaded and saved to /tmp/tmpqdsytb_b
0/1: /tmp/tmpqdsytb_b pre-prossessing...


<string>:4: UserWarning: PySoundFile failed. Trying audioread instead.
/home/j-j11a210/.local/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
I0000 00:00:1728601406.576955 1629003 service.cc:146] XLA service 0x7f48c0004b40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728601406.576998 1629003 service.cc:154]   StreamExecutor device (0): Tesla V100-PCIE-32GB, Compute Capability 7.0
2024-10-11 08:03:26.583105: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-11 08:03:26.604672: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90300


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 324ms/step


...

10:44:39.667948 {'noteId': 52, 'objectUrl': 'https://timeisnullnull.s3.ap-northeast-2.amazonaws.com/wav/1b495c40-1e40-412a-9846-a24039ee0e30.wav', 'processId': 'e82397b1-bb28-47bc-a415-48c8fccec3ea'}
10:44:39.668273 Downloading audio file from URI: https://timeisnullnull.s3.ap-northeast-2.amazonaws.com/wav/1b495c40-1e40-412a-9846-a24039ee0e30.wav


INFO:__main__:10:44:39.890488  Audio file downloaded and saved to /tmp/tmpnfp1uouh
INFO:werkzeug:127.0.0.1 - - [11/Oct/2024 10:44:39] "POST /voice/analysis HTTP/1.1" 202 -
<string>:4: UserWarning: PySoundFile failed. Trying audioread instead.
/home/j-j11a210/.local/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


10:44:39.890404 Audio file downloaded and saved to /tmp/tmpnfp1uouh
0/1: /tmp/tmpnfp1uouh pre-prossessing...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
[32.78, 32.79, 32.8, 32.81]
10:44:40.400702 request: {'id': 52, 'process_id': 'e82397b1-bb28-47bc-a415-48c8fccec3ea', 'anomaly_time': [32.78, 32.79, 32.8, 32.81]}
10:44:40.400749 responst: <Response [200]>


INFO:__main__:10:54:08.101152 Downloading audio file from URI: https://timeisnullnull.s3.ap-northeast-2.amazonaws.com/wav/f62af019-07c9-461a-bf99-ad6c2bae6252.wav
INFO:__main__:10:54:08.293503  Audio file downloaded and saved to /tmp/tmpdbw6pwym
<string>:4: UserWarning: PySoundFile failed. Trying audioread instead.
/home/j-j11a210/.local/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
INFO:werkzeug:127.0.0.1 - - [11/Oct/2024 10:54:08] "POST /voice/analysis HTTP/1.1" 202 -


10:54:08.100425 {'noteId': 52, 'objectUrl': 'https://timeisnullnull.s3.ap-northeast-2.amazonaws.com/wav/f62af019-07c9-461a-bf99-ad6c2bae6252.wav', 'processId': '3cb88707-23b7-4696-95cb-dbd89e3c1de4'}
10:54:08.101130 Downloading audio file from URI: https://timeisnullnull.s3.ap-northeast-2.amazonaws.com/wav/f62af019-07c9-461a-bf99-ad6c2bae6252.wav
10:54:08.293439 Audio file downloaded and saved to /tmp/tmpdbw6pwym
0/1: /tmp/tmpdbw6pwym pre-prossessing...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
[]
10:54:08.676241 request: {'id': 52, 'process_id': '3cb88707-23b7-4696-95cb-dbd89e3c1de4', 'anomaly_time': []}
10:54:08.676289 responst: <Response [200]>
